# Converting MATLAB wbstructs to pandas dataframes

Check the README.md for more information when to use this tool. This notebook simply calls the functions in the wbstruct_converter.py file. To get a better understanding of the code, please refer to the wbstruct_converter.py file. If there are unhandled errors, please open an issue on GitHub.

In [ ]:
import utils.wbstruct_to_dicts as wbstruct_dictioniaries
import utils.wbstruct_dicts_to_dataframes as wbstruct_dataframes
import dill
from collections import defaultdict
import numpy as np
import pandas as pd



# in case we want to reload our module
import importlib
importlib.reload(wbstruct_dictioniaries)
importlib.reload(wbstruct_dataframes)

### Extracting Rebecca's data

deltaFOverF_bc (bleach corrected version), we might have to visit the non-bleach corrected version as well though

In [ ]:
data_source = "Y:\\lisc\\project\\neurobiology\\zimmer\\Rebecca\\Analyses\\Whole_WT_Analyses"

In [ ]:
# import and save rebecca's data 
files = wbstruct_dictioniaries.find_file(data_source,"WholeWTData.mat","","")
matfile = wbstruct_dictioniaries.load_matlab_file(files[0]) 

In [ ]:
datasets = defaultdict(pd.DataFrame)

count = 0

for dataset in matfile['WholeWTData']['dataset'][0]:
    ds = dataset[0].tolist()
    key = ds.split("_")[2]+"_"+ds.split("_")[3]
    # merging head and tail data
    trialdf = matfile["WholeWTData"]["deltaFOverF_bc"][0][count]

    id_names = wbstruct_dataframes.get_values(matfile['WholeWTData']['ID'][0][count][0])
    for i in range(len(id_names)):
        
        if isinstance(id_names[i], np.ndarray):
            id_names[i] = wbstruct_dataframes.get_values(id_names[i][0])
        if isinstance(id_names[i], list):
            id_names[i] = id_names[i][0]
    
    id_length = trialdf.shape[1]
    colnames = [f"neuron_{i:03d}" for i in range(id_length)]
    colnames = [dummy if pd.isna(ID) else ID for dummy, ID in zip(colnames, id_names)]
    newdf = pd.DataFrame(trialdf, columns=colnames)
    avar = newdf.columns.get_loc("AVAR")
    avar_traces = matfile['WholeWTData']["traceColoring"][0][count][:, avar]
    aval = newdf.columns.get_loc("AVAL")
    aval_traces = matfile['WholeWTData']["traceColoring"][0][count][:, aval]
    if 0 in avar_traces:
        newdf["state"] = aval_traces
    else:
        newdf["state"] = avar_traces
    datasets[key] = newdf
    count = count + 1
    

In [ ]:
with open('../preprocessing/dataframes_rebecca_2301.pkl','wb') as f:
    dill.dump(datasets, f)